## Try-except en cada parte para manejo de errores

In [0]:
# DBTITLE 1: Carga de parámetros desde secret scope
try:
  event_hub_conn_str = dbutils.secrets.get(scope="scopeTransaction", key="keyTransaction")
  bronze_path = "abfss://bronzesmartbank@mistorageprincipal.dfs.core.windows.net/transactions_eventhub/"

  dbutils.secrets.list("scopeTransaction")
#
  storage_account_name = "mistorageprincipal"
  storage_account_key = dbutils.secrets.get("scopeTransaction", "keyTransaction")

  spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
  )
except Exception as e:
    dbutils.notebook.exit(f"[ERROR - BRONZE INIT] {str(e)}")


In [0]:
dbutils.fs.ls(bronze_path)

In [0]:
# DBTITLE 1: Crear catálogo y esquema (si no existen)

# spark.sql("CREATE SCHEMA IF NOT EXISTS bronze_transactions.tables")
# spark.sql(
#         CREATE TABLE IF NOT EXISTS capa_bronze.tables.{name} AS
#         SELECT * FROM {name}
# """)



In [0]:
# DBTITLE 1: Configuración inicial
from pyspark.sql.types import StructType, StringType, DoubleType
from pyspark.sql.functions import col


try:
# Leer secret del container de destino (si usás conexión segura, opcional)
    dbutils.secrets.get(scope="scopeTransaction", key="keyTransaction")

# Ruta de origen y destino  
    source_path = "abfss://rawsmartbank@mistorageprincipal.dfs.core.windows.net/transacciones/"

# Esquema de las transacciones
    transaction_schema = StructType() \
        .add("transaction_id", StringType()) \
        .add("customer_id", StringType()) \
        .add("amount", DoubleType()) \
        .add("timestamp", StringType()) \
        .add("merchant", StringType()) \
        .add("country", StringType()) \
        .add("channel", StringType()) \
        .add("category", StringType())

# Lectura con Autoloader desde el contenedor RAW
    raw_df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .schema(transaction_schema)
        .load(source_path)
    )

# Escritura en capa Bronze con Delta Lake
    (
        raw_df.writeStream
        .format("delta")
        .outputMode("append")
        .option("checkpointLocation", bronze_path + "/_checkpoint")
        .trigger(once=True)
        .start(bronze_path)
        .awaitTermination()
    )

except Exception as e:
    dbutils.notebook.exit(f"[ERROR - BRONZE WRITE] {str(e)}")

dbutils.notebook.exit("OK") 
